## <center><strong>Notebook<span style= "color: #51FCC6"> Describtion

## <center><strong>Importing<span style= "color: #48E0DC"> Packeges

In [1]:
import time
import sqlite3
import warnings
import requests
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
from itertools import count
from bs4 import BeautifulSoup
from selenium import webdriver
import matplotlib.pyplot as plt
from selenium.webdriver.common.by import By
from IPython.display import set_matplotlib_formats
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
COLORS = ["#51fcc6", "#48e0dc", "#5cd3f7", "#4895e0", "#517afc"]

%matplotlib inline

NUMERICS = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64',
            'uint16', 'uint32', 'uint64']

set_matplotlib_formats('pdf', 'svg')
warnings.filterwarnings("ignore")

C:\Users\FreeComp\AppData\Local\Temp\ipykernel_2904\2138414856.py:8: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('pdf', 'svg')


## <center><strong>Setting up the<span style= "color: #5CD3F7"> Web scrapers

In [4]:
data_jobs_titles = ["data-entry", "data-processing", "data-engineering",
                    "data-science", "data-analytics", "data-visualization"]

upwork_types = ["profiles", "jobs", "services"]
freelancer_types = ["users", "projects"]
peapleperhour_types = ["hire-freelancers", "freelance-jobs", "services"]
guru_service_types = ["freelancers", "jobs"]


def scrape_page(url: str) -> BeautifulSoup:
    driver = webdriver.Chrome()
    driver.get(url)
    
    wait = WebDriverWait(driver, 10)
    
    # Wait until the page is fully loaded
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    
    html = driver.page_source
    driver.quit()
    
    return BeautifulSoup(html)

In [5]:
# The `type` param indicates what is the nature of the job is it hiring freelancers 
# or paying services or othr thing based on the platform.

def upwork_scraper(page, job_title, type):

    global upwork_url

    if  type == 'services':

        upwork_url =  f"https://www.upwork.com/services/search?nbs=1&q={job_title}"

        driver = webdriver.Chrome()
        driver.get(upwork_url)

        if page > 1:
            for i in ragne(page):
                
                wait = WebDriverWait(driver, 10)
                element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.up-btn.up-btn-default.mx-0.load-more-btn.nuxt-link-active')))
                element.click()
                
                time.sleep(1)

        html = driver.page_source
        driver.quit()
        
        return  html

    elif type == "profiles" or type == "jobs":
        
        upwork_url = f"https://www.upwork.com/search/{type}/?page={page}&q={job_title}&user_pref=1"
        
        return scrape_page(upwork_url)


def guru_scraper(page, job_title, type):

    global guru_url
    guru_url = f"https://www.guru.com/d/{type}/skill/{job_title}/pg/{page}/"

    return scrape_page(guru_url)


def fivver_scraper(page, job_title):

    global fivver_url
    fivver_url = f"https://www.fiverr.com/categories/data/{job_title}?source=pagination&page={page}"

    return scrape_page(fivver_url)


def peopleperhour_scraper(page, job_title, type):

    global peopleperhour_url

    if type == "hire-freelancers":
        peopleperhour_url = f"https://www.peopleperhour.com/hire-freelancers/{job_title.replace('-', '+')}?page={page}&ref=search"

    else:
        peopleperhour_url = f"https://www.peopleperhour.com/{type}-{job_title.replace('-', '+')}"

    return  scrape_page(peopleperhour_url)

# We will not analyze requested jobs data in freelancer.com becuase it's search feature 
# isn't accurate with the it.

def freelancer_scraper(page, job_title, type):

    global freelancer_url
    freelancer_url = f"https://www.freelancer.com/search/{type}?q={job_title.replace('-', '%20')}&page={page}"
    
    return scrape_page(freelancer_url)

Now we will try to calculate how much did each scraper take to collect the data

In [6]:
get_variable_name = lambda var: [name for name in globals() if globals()[name] is var][0]

def get_runtime(func, *args, **kwargs):

    t1 = time.perf_counter()
    output = func(*args, **kwargs)
    t2 = time.perf_counter()

    run_time = t2 - t1

    print("{} took about {:.2f}s to run.\n".format(
        get_variable_name(func).replace("_", " ").title(), run_time))

    return  output

Now we will just try to scrape **only one page** from each scraper *(temporary action)* to just test the data extractors.

## <center><strong>Extracting jobs <span style = "color: #4895e0"> Links</span>

In [21]:
scrape_page("https://muhammed-abdelaleam.github.io/me/")

<html lang="en"><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<!-- css -->
<link href="css/all.min.css" rel="stylesheet"/>
<link href="css/framework.css" rel="stylesheet"/>
<link href="css/master.css" rel="stylesheet"/>
<!-- open graph -->
<!-- HTML Meta Tags -->
<title>Muhammed Ahmed Abd Al-Aleam</title>
<meta content="Experienced data analyst with expertise in statistical analysis, data visualization, and machine learning. Skilled in Python and SQL. Passionate about using data to drive business decisions" name="description"/>
<link href="assets/logo_v2.ico" rel="shortcut icon"/>
<!-- Facebook Meta Tags -->
<meta content="https://muhammed-abdelaleam.github.io/me" property="og:url"/>
<meta content="website" property="og:type"/>
<meta content="Muhammed Ahmed Abd Al-Aleam" property="og:title"/>
<meta content="Experienced data analyst with expertise in statistical analysis, data visualization, and machine learning. Skilled in Python

In [7]:
soups = {"soup_name": [],
         "soup_job_type": [],
         "soup_job_title": [],
         "soup_page":  [],
         "soup": []}

In [19]:
# If the job type is services in upwork the pages paremeter will refer to how many pages 
# will render at once while the other scrapers will render the spesfic spesfied page only.

def get_all_pages_soups(pph_type, freelancer_type, upwork_type, guru_type, max_page: int= 999) -> dict:

    print("Strating ...")
    for i in count(0):

        global orgnaize_data_in_dict
        page = i + 1

        print(f"Loading page {page} of each website ...")

        for data_job_title in data_jobs_titles:

            lines = [f'peopleperhour_html = peopleperhour_scraper({page}, "{data_job_title}", "{pph_type}")',
                     f'fivver_html = fivver_scraper({page}, "{data_job_title}")',
                     f'freelancer_html = freelancer_scraper({page}, "{data_job_title}", "{freelancer_type}")',
                     f'upwork_html = upwork_scraper({page}, "{data_job_title}", "{upwork_type}")',
                     f'guru_html = guru_scraper({page}, "{data_job_title}", "{guru_type}")']

            for line_idx, line in enumerate(lines):

                equall_sign_idx = line.find('=')

                def orgnaize_data_in_dict():
                    soups["soup_job_title"].append(data_job_title)
                    soups["soup_name"].append(line[:equall_sign_idx])
                    soups["soup_page"].append(page)
                    soups["soup"].append(exec(line[:equall_sign_idx]))
                    soups["soup_job_type"].append(job_type)

                try:
                    print(line)
                    exec(line)
                    orgnaize_data_in_dict()


                except Exception as e:

                    print(e)
                    del lines[line_idx]
                    if len(lines) < 1 or page >= max_page:

                        print("Finished ...")
                        return  soups
                        break

                    continue

In [23]:
get_all_pages_soups("freelance-jobs", "projects", "jobs", "jobs", 1)

Strating ...
Loading page 1 of each website ...
peopleperhour_html = peopleperhour_scraper(1, "data-entry", "freelance-jobs")
name 'peopleperhour_html' is not defined
Finished ...


{'soup_name': ['peopleperhour_html ',
  'freelancer_html ',
  'peopleperhour_html ',
  'peopleperhour_html '],
 'soup_job_type': [],
 'soup_job_title': ['data-entry', 'data-entry', 'data-entry', 'data-entry'],
 'soup_page': [1, 1, 1, 1],
 'soup': []}

In [9]:
dict = {"col1": [1, 2, 3, 5, 5],
        "col2": [1, 2, 3, 9]}

max_length = max(len(lst) for lst in dict.values())
dict = {key: lst + [float('nan')] * (max_length - len(lst)) for key, lst in dict.items()}

dict

{'col1': [1, 2, 3, 5, 5], 'col2': [1, 2, 3, 9, nan]}

In [8]:
print("Printing the title of each page to check if it's loaded correctly.\n")

for soup in soups["soup"]:
    print("-\t", soup.title.text.replace("\n", "").replace("\t", ""))

soups = pd.DataFrame(soups)

Printing the title of each page to check if it's loaded correctly.

-	 Freelance Data-analytics Jobs - Upwork
-	 Data Analytics Services Online | Fiverr
-	 Machine Learning Freelance Jobs Online - Guru
-	 Get Data Analytics Offers & Services | PeoplePerHour
-	 Users Jobs, Employment | Freelancer


Looks like everything is working as expected now let's start reel work by extracting the HTML tags data from the scraped<br>
pages.<br> <br>

But we will need to collect the data from each one of those pages with unique way further more we need to optimize the<br>
data extractor to collect the data from other pages from the same site like "jobs" site and "profiles" sites on upwork.<br>

In [67]:
def get_cards_links(soup_name, tag, link_template,
                    link_attr, tag_class, attr= "class") -> list:

    soup = soups[soups["soup_name"] == soup_name]["soup"].values[0]

    cards = soup.find_all(tag, attrs= {attr: tag_class})
    cards_ids = []

    if link_attr != "":

        for card in cards:

            card_id = link_template.replace("###", card[link_attr])

            if len(card_id) > len(link_template):
                cards_ids.append(card_id)

        return  cards_ids

    else:
        return  cards

In [85]:
# We can't collect guru data this wat becuase it has slightly deffrint case in terms
# of collecting the jobs URLs

## --------------------------Upwork--------------------------
upwork_gigs_links = get_cards_links('upwork_soup', 'a', 'https://upwork.com###', 'href',
                                        'up-n-link project-tile up-card m-0 p-0 up-card-section up-card-hover project-tile_flexible')

upwork_profiles_links = get_cards_links('upwork_soup', 'div', 'https://www.upwork.com/search/profiles/?profile=###', 'data-test-key',
                                         'up-card-section up-card-hover')

upwork_jobs_parents = get_cards_links('upwork_soup', 'h3', 'https://www.upwork.com###', '', 'my-0 p-sm-right job-tile-title')
upwork_jobs_links = []

for parent in upwork_jobs_parents:
    upwork_jobs_links.append("https://www.upwork.com" + parent.find('a', class_= "up-n-link")['href'])
## ----------------------------------------------------------


## --------------------------Fivver--------------------------
fivver_gigs_links = get_cards_links('fivver_soup', 'a', 'https://www.fiverr.com###', 'href', 'tbody-5 p-t-4 YLycza2 u9KHmsf')
## ----------------------------------------------------------


## ---------------------People per hour----------------------
pph_services_links = get_cards_links("peopleperhour_soup", "a", "https://www.peopleperhour.com###", "href", 
                                     "card__title-wrapper⤍FreelancerCard⤚2-NWk")

pph_freelancers_links = get_cards_links("peopleperhour_soup", "a", "https://www.peopleperhour.com###", "href",
                                        "card__title-wrapper⤍FreelancerCard⤚2-NWk")

pph_jobs_links = get_cards_links("peopleperhour_soup", "a", "https://www.peopleperhour.com###", "href",
                                 "item__url⤍ListItem⤚20ULx")
## ----------------------------------------------------------


## ------------------------Guru------------------------------
guru_cards_parents_profiles = get_cards_links("guru_soup", "h2", "https://guru.com###", "",
                                              "jobRecord__title jobRecord__title--changeVisited")

guru_cards_parents_jobs = get_cards_links("guru_soup", "h2", "https://guru.com###", "",
                                          "jobRecord__title jobRecord__title--changeVisited")
## ----------------------------------------------------------


## ----------------------Freelancer---------------------------
freelancer_jobs_links = get_cards_links("freelancer_soup", "a", "https://www.freelancer.com###", "href",
                                        "LinkElement ng-star-inserted")

freelancer_freelancers_links = get_cards_links("freelancer_soup", "a", "https://www.freelancer.com###", "href",
                                              "LinkElement ng-star-inserted")

In [91]:
freelancer_freelancers_links

[]

In [ ]:
def get_upwork_links(job_type: str) -> list:  

    card_tag = "a"
    link_template = "https://www.upwork.com###"
    link_attr = "href"

    if job_type == "profiles":

        card_tag = "div"
        card_tag_class = "up-card-section up-card-hover"
        link_template = "https://www.upwork.com/search/profiles/?q=profiles&profile=###"
        link_attr = "data-test-key"
    
    elif job_type == "jobs":
        card_tag_class = "up-n-link"


    elif job_type == "services":
        card_tag_class = "up-n-link project-tile up-card m-0 p-0 up-card-section up-card-hover project-tile_flexible"

    else:
        raise Exception("Please input correct work type from ['profiles', 'jobs', 'services'].")


    soup = soups[soups["soup_name"] == "upwork_soup"]["soup"][0]

    cards = soup.find_all(card_tag, class_= card_tag_class)
    cards_ids = []

    for card in cards:
        
        card_id = link_template.replace("###", card[link_attr])
        
        if len(card_id) > len(link_template):
            cards_ids.append(card_id)

    return  cards_ids


def get_fivver_links() -> list:

    card_tag = "div"
    card_tag_class = "basic-gig-card"
    link_template = "https://www.fiverr.com###"
    link_attr = "href"

    cards = soup.find_all(card_tag, class_= card_tag_class)
    cards_ids = []
    
    for card in cards:
        
        card_id = link_template.replace("###", card[link_attr])
        
        if len(card_id) > len(link_template):
            cards_ids.append(card_id)

    return  cards_ids

In [ ]:
def get_guru_links(job_type: str) -> list:  

    soup = soups[soups["soup_name"] == "guru_soup"]["soup"][2]
    link_template = "https://www.guru.com###"

    if job_type == "freelancers":
        cards_parents = soup.find_all(
            'h2', class_= 'serviceListing__title serviceListing__title--dark')

        cards = []

        for parent in cards_parents:
            card = parent.find('a')
            cards.append(card)

    elif job_type == "jobs":
        cards_parents = soup.find_all(
            'h2', class_ = 'jobRecord__title jobRecord__title--changeVisited')

        cards = []

        for parent in cards_parents:
            card = parent.find('a')
            cards.append(card)

    cards_ids = []

    for card in cards:
        card_id = link_template.replace("###", card['href'])
        cards_ids.append(card_id)

    return  cards_ids

In [ ]:
get_guru_links('jobs')